In [37]:
import json
import pyrebase 
from urllib.parse import quote


# Monkey patch pyrebase: replace quote function in pyrebase to workaround a bug.
# See https://github.com/thisbejim/Pyrebase/issues/294.
pyrebase.pyrebase.quote = lambda s, safe=None: s


# Monkey patch pyrebase: the Storage.get_url method does need quoting :|
def get_url(self, token=None):
    path = self.path
    self.path = None
    if path.startswith('/'):
        path = path[1:]
    if token:
        return "{0}/o/{1}?alt=media&token={2}".format(self.storage_bucket, quote(path, safe=''), token)
    return "{0}/o/{1}?alt=media".format(self.storage_bucket, quote(path, safe=''))


pyrebase.pyrebase.Storage.get_url = lambda self, token=None: \
    get_url(self, token)

In [38]:
# firebase config
with open("fb_config.json") as jsonfile:
    # `json.loads` parses a string in json format
    config_info = json.load(jsonfile)
    FB_CONFIG = config_info['FB_CONFIG']


# firebase init
firebase = pyrebase.initialize_app(FB_CONFIG)
storage = firebase.storage()
auth = firebase.auth()
db = firebase.database()


In [39]:
with open("db.json") as jsonfile:
    # `json.loads` parses a string in json format
    config_info = json.load(jsonfile)
    tests = config_info['tests']
    tests_meta = config_info['tests_meta']    

In [6]:
tests_meta

{'1': {'test_id': 'acbd14e370c0452a82f88e5104c92dba',
  'video_file_name': 'V_20210512_162548_vHDR_On_N0.mp4',
  'datetime': '25/05/2021 11:41:01',
  'fps': 29.79599938911539,
  'athlete': 'MarinaBulbarella',
  'blink_threshold': 3.5,
  'bf_timestep': 10}}

In [18]:
db.child("tests").push(tests)


{'name': '-Ma_cRXMO8FOBxMpATmD'}

In [19]:
db.child("tests_meta").push(tests_meta)
db.child("tests_meta").child("Morty").set(tests_meta)

{'name': '-Ma_cSiZ_HygZyzBBxgd'}

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore
# initialize sdk
cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred)
# initialize firestore instance
firestore_db = firestore.client()

In [17]:
# add data
firestore_db.collection(u'tests_meta').add(tests_meta)

(DatetimeWithNanoseconds(2021, 5, 26, 0, 1, 36, 439096, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7fe391c74e20>)

In [16]:
# read data
tests = list(firestore_db.collection(u'tests_meta').get())
for test in tests:
    print(test.to_dict())

{'1': {'video_file_name': 'V_20210512_162548_vHDR_On_N0.mp4', 'datetime': '25/05/2021 11:41:01', 'blink_threshold': 3.5, 'test_id': 'acbd14e370c0452a82f88e5104c92dba', 'athlete': 'MarinaBulbarella', 'bf_timestep': 10, 'fps': 29.79599938911539}}


In [77]:
metrics = db.child('1-baN5lUPSQnLS3NdHV86EYFlrn_FZkyvjIfalsTmrdM').order_by_child("FECHA").child('MARINA').get()
#.child('leo mayer').get()

# metrics_dict = {}
# for metric in metrics.each():
#     metrics_dict[metric.key()]= metric.val()

# metrics_dict

# sorted([datetime.strptime(j['FECHA'], '%d-%m-%y') for i, j in enumerate(metrics.val())], )
key_value = [j for j in metrics.val()]
a = sorted(key_value, key = lambda x:x['FECHA'], reverse=True)

[i["ELO"] for i in a]
# key_value.keys()
# data = sorted([i['FECHA'] for i in metrics.val()])
# key_value.items()
# # elo = data['ELO']
# elo

# data
# metrics.val()

['67', '45', '131', '34', '91', '200', '234']

In [87]:
    map_values = {
        "0": u"\U0001F7E2", #green
        "1": u'\U0001F7E2',
        "2": u'\U0001F7E2',
        "3": u'\U0001F7E1', #yellow
        "4": u'\U0001F7E1',
        "5": u'\U0001F7E1',
        "6": u'\U0001F534', #red
        "7": u'\U0001F534',
        "8": u'\U0001F534',
        "9": u'\U0001F534'	
    }

In [118]:
data = db.child('1-baN5lUPSQnLS3NdHV86EYFlrn_FZkyvjIfalsTmrdM').child('MARINA').get()

_data = [j for j in data.val()]
sorted_data = sorted(_data, key = lambda x: x['FECHA'], reverse=True)


cuadrantes = ["a1", "a2", "a3", "b1", "b2", "b3", "c1", "c2", "c3"]

initial_string = 'resistencia_ocular_pestaneo_x_cuadrante'

map(lambda x,y: zip(x,y), [{cuadrante: test[f'{initial_string}_{cuadrante}'] for cuadrante in cuadrantes} for test in sorted_data])

TypeError: <lambda>() missing 1 required positional argument: 'y'

In [7]:
a = db.child('1-baN5lUPSQnLS3NdHV86EYFlrn_FZkyvjIfalsTmrdM').child('LOS ANDES').order_by_child('POSICION').get()

posiciones = set(val for sublist in [j['POSICION'].split(', ') for i,j in metric.val().items()] for val in sublist)

[i for i,j in db.child('1-baN5lUPSQnLS3NdHV86EYFlrn_FZkyvjIfalsTmrdM').child('LOS ANDES').order_by_child('POSICION').equal_to('arquero').get().val().items()]

NameError: name 'metric' is not defined

In [140]:
arqueros = [item for item in a[1:] if 'arquero' in item['posicion']]
arqueros
# lopez[0]['FECHA de evaluación'][:10]

[{'BB ': '',
  'CADERA': 'D ',
  'DEPORTE': 'futbol',
  'DRIVE': '',
  'ELO': 13,
  'FECHA de evaluación': '2021-03-15T03:00:00.000Z',
  'FOTO ': '',
  'HC': '',
  'HOMBRO': 'I ',
  'ID': '00102F',
  'NOMBRE': 'Leandro leonel, Brey',
  'OTAC': 'D ',
  'OTEC': 'I',
  'OVERALL': '',
  'PIE DE PATEO': 'D ',
  'PIERNA DE APOYO ': 'I ',
  'PIERNA DINAMICA': 'D ',
  'TIPO TEST': 'EI',
  'i': 1,
  'posicion': 'arquero'},
 {'BB ': '',
  'CADERA': 'D ',
  'DEPORTE': 'futbol',
  'DRIVE': '',
  'ELO': 3,
  'FECHA de evaluación': '2021-03-15T03:00:00.000Z',
  'FOTO ': '',
  'HC': '',
  'HOMBRO': 'I ',
  'ID': '00103F',
  'NOMBRE': 'Martin Guillermo, Soria',
  'OTAC': 'D ',
  'OTEC': 'I',
  'OVERALL': '',
  'PIE DE PATEO': 'D ',
  'PIERNA DE APOYO ': 'I ',
  'PIERNA DINAMICA': 'D ',
  'TIPO TEST': 'EI',
  'i': 2,
  'posicion': 'arquero'},
 {'BB ': '',
  'CADERA': 'I ',
  'DEPORTE': 'futbol',
  'DRIVE': '',
  'ELO': 41,
  'FECHA de evaluación': '2021-03-15T03:00:00.000Z',
  'FOTO ': '',
  'HC': '',


In [136]:
from datetime import datetime

def str_to_dt(str):
    datetime_object = datetime.strptime(str[:10], '%Y-%m-%d')
    return datetime_object

str_to_dt(lopez[0]['FECHA de evaluación'][:10])

datetime.datetime(2021, 3, 15, 0, 0)

In [137]:
{'ELO': {item['FECHA de evaluación'][:10]: item['ELO'] for item in lopez}}

{'ELO': {'2021-03-15': 31, '2021-04-02': 8}}

In [96]:
from dicttable import DictTable

DB = DictTable(items)

[i for i in DB.query(DB.Q.HOMBRRO=='D')]

[]